# 해시태그로 검색 후, 인기 게시글 정보 가져오기

* 전달받은 데이터셋 : 정제되지 않은 상품 이름 데이터

* 작업 목적 : 특정 상품이 해시태그로 걸린 피드를 검색하고, 이 중 인기 게시글만 가져옴

- 인스타그램은 json형식의 데이터도 제공해주므로 beautiful soup/ selenium  대신 requests를 사용함

In [1]:
import requests
import json
import pandas as pd
import random
import time
import re
from tqdm import tqdm
from datetime import datetime


# 인스타그램 해시태그 규격에 맞는 검색어를 보내기 위한 사전작업
# 영어,숫자를 제외한 문자는 모두 제거
# 공백 모두 제거
def hashtag_text(x):
    hashtag = re.sub('[^a-zA-Z0-9]','',x).strip()
    return hashtag

In [4]:
# 위의 함수 예시
print(product_crawl_list['product_name'].iloc[20])
print(hashtag_text(product_crawl_list['product_name'].iloc[20]))

Norvina® Mini Pro Pigment Palette Vol. 3
NorvinaMiniProPigmentPaletteVol3


In [6]:
data_list = []
error_list = []
rest = random.randint(10,12)

product_crawl_list = pd.read_excel('product_glami_crawl_list.xlsx')
product_crawl_list['hashtag'] = product_crawl_list['product_name'].apply(hashtag_text)

today = datetime.today().strftime("%m%d")

for idx,product_list in tqdm(product_crawl_list[0:len(product_crawl_list)].iterrows(), desc="돌리는중"):
    
    try :
        
        tag = glami_list['hashtag']
        product_code = product_list['product_code']
        product_name = product_list['product_name']
        #print(tag)
        
        tag_url = f'https://www.instagram.com/explore/tags/{tag}/?__a=1'
        headers = {
                'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'
                  }

        sess = requests.Session()
        response = requests.get(tag_url, headers=headers)
        time.sleep(rest)
        result = json.loads(response.text)
        edges = result['graphql']['hashtag']['edge_hashtag_to_top_posts']['edges']

        for edge in edges:

            #hashtag = tag
            url = tag_url
            peed_url = 'https://www.instagram.com/p/' + edge['node']['shortcode']
            img_url = peed_url + '/media/?size=l'
            data = {'product_code':product_code,'product_name':product_name,
                    'search_query':tag ,'instagram_url':peed_url,'image_url' : img_url }
            data_list.append(data)
    
    except :
        
        error = {'product_code':product_code,'product_name':product_name,'search_query':tag}
        error_list.append(error)
        #print('error page - ! or None')

    if (idx+1) % 50 == 0 :
        
        time.sleep(50)

        
product_popular_peed = pd.DataFrame(data_list,columns=['product_name','search_query','instagram_url','image_url'])
product_none_peed = pd.DataFrame(error_list,columns=['product_name','search_query'])


url_preserve = pd.ExcelWriter(f'product_{today}_3.xlsx', options={'strings_to_urls': False})
product_popular_peed.to_excel(url_preserve)
url_preserve.save()

# 해당 해시태그 게시물이 존재하지 않는 에러 데이터들
if len(product_none_peed) != 0:
    product_none_peed.to_excel(f"prodcut_insta_crawling_error_{today}.xlsx")
    
    

돌리는중: 5it [00:55, 11.12s/it]


In [9]:
# 최총 수집 데이터
product_popular_peed.head(10)

,product_name,search_query,instagram_url,image_url
0,Norvina® Mini Pro Pigment Palette Vol. 3,NorvinaMiniProPigmentPaletteVol3,https://www.instagram.com/p/CAyDx0gILrK,https://www.instagram.com/p/CAyDx0gILrK/media/...
1,Norvina® Mini Pro Pigment Palette Vol. 3,NorvinaMiniProPigmentPaletteVol3,https://www.instagram.com/p/B-UsEEenwVA,https://www.instagram.com/p/B-UsEEenwVA/media/...
2,Metropolis Eyeshadow Palette,MetropolisEyeshadowPalette,https://www.instagram.com/p/CA0WWfGnCb4,https://www.instagram.com/p/CA0WWfGnCb4/media/...
3,Metropolis Eyeshadow Palette,MetropolisEyeshadowPalette,https://www.instagram.com/p/CBJXzHKHQF3,https://www.instagram.com/p/CBJXzHKHQF3/media/...
4,Metropolis Eyeshadow Palette,MetropolisEyeshadowPalette,https://www.instagram.com/p/CA0wkglHYW0,https://www.instagram.com/p/CA0wkglHYW0/media/...
5,Metropolis Eyeshadow Palette,MetropolisEyeshadowPalette,https://www.instagram.com/p/CA0B7YdHNpm,https://www.instagram.com/p/CA0B7YdHNpm/media/...
6,Metropolis Eyeshadow Palette,MetropolisEyeshadowPalette,https://www.instagram.com/p/B_AtCG9HDi5,https://www.instagram.com/p/B_AtCG9HDi5/media/...
7,Metropolis Eyeshadow Palette,MetropolisEyeshadowPalette,https://www.instagram.com/p/CBWatuzH0fq,https://www.instagram.com/p/CBWatuzH0fq/media/...
8,Metropolis Eyeshadow Palette,MetropolisEyeshadowPalette,https://www.instagram.com/p/CBLk0xVHWS3,https://www.instagram.com/p/CBLk0xVHWS3/media/...
9,Metropolis Eyeshadow Palette,MetropolisEyeshadowPalette,https://www.instagram.com/p/CBYLq31H6nE,https://www.instagram.com/p/CBYLq31H6nE/media/...
